In [1]:
import os
if(not os.getcwd().endswith("crchum")):
    os.chdir("..")
os.getcwd()

'/Users/antoinemagron/Documents/EPFL/PDM/crchum'

In [2]:
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

In [12]:
from transformers import (AutoModel, AutoTokenizer, LlamaForCausalLM)

In [16]:
tok = AutoTokenizer.from_pretrained("epfl-llm/meditron-7b")
meditron = LlamaForCausalLM.from_pretrained("epfl-llm/meditron-7b")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [17]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [18]:
meditron = meditron.to(mps_device)
answer = meditron.generate(
    **tok("My patient is epileptic.", return_tensors="pt").to(mps_device)
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [22]:
tok.batch_decode(answer, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'My patient is epileptic.\n\n### 1.\nWhat is the most'

<ins>Generation time</ins> : 20 min for one answer 